In [1]:
import pandas as pd

In [2]:
authors = ['Dickens, Charles', 'Collins, Wilkie', 'Scott, Walter', 'Meredith, George', 'Conrad, Joseph', 'Cather, Willa'] 
database = 'sqlite:////home/jon/Code/gitenberg-scrape/pg-text-7.db'

In [3]:
def getMeta(author): 
    query = 'select * from meta where author="%s"' % author
    meta = pd.read_sql(query, database)
    return meta

In [4]:
dickensMeta = getMeta(authors[0])

In [5]:
dickensNovelIDs = [3, 13, 34, 12, 28, 35, 19, 14, 36, 15, 33, 1, 40, 23, 2]

In [6]:
dNovelsMeta = dickensMeta.loc[dickensNovelIDs][['title', 'gr_pubDate', 'id']]

In [7]:
dickensBib = pd.read_csv('dickens.csv')

In [139]:
dNovelsMeta = dNovelsMeta.reset_index()
dNovelsMeta['wp_pubDate'] = dickensBib['Year completed']

In [8]:
def getText(id):
    query = 'select text from text where id="%s"' % id
    result = pd.read_sql(query, database)
    if len(result) > 0: 
        return result.loc[0].text
    else:
        return None

In [133]:
dNovelsMeta['text'] = dNovelsMeta['id'].apply(getText)

In [134]:
dNovelsMeta.loc[15]

title         Hard Times
gr_pubDate        1854--
id                 786.0
wp_pubDate           NaN
text                None
Name: 15, dtype: object

In [135]:
dickensMeta[dickensMeta.title=='Hard Times'].title

15    Hard Times
Name: title, dtype: object

In [113]:
dNovelsMeta = dNovelsMeta.reset_index()

In [137]:
hardTimes = open('/home/jon/Code/gitenberg-scrape/hard-times.txt').read()
dNovelsMeta.set_value(15, 'text', hardTimes)

                                            title gr_pubDate      id  \
3                             The Pickwick Papers     1837--   580.0   
13                                   Oliver Twist     1838--   730.0   
34                              Nicholas Nickleby     1839--   967.0   
12                         The Old Curiosity Shop     1840--   700.0   
28  Barnaby Rudge: A Tale of the Riots of 'Eighty         --   917.0   
35                              Martin Chuzzlewit     1844--   968.0   
19                                 Dombey and Son     1848--   821.0   
14                              David Copperfield     1849--   766.0   
36                                    Bleak House     1853--  1023.0   
15                                     Hard Times     1854--   786.0   
33                                  Little Dorrit     1855--   963.0   
1                            A Tale of Two Cities     1859--    98.0   
40                             Great Expectations     1860--  14

In [140]:
dNovelsMeta

    index                                          title gr_pubDate      id  \
0       3                            The Pickwick Papers     1837--   580.0   
1      13                                   Oliver Twist     1838--   730.0   
2      34                              Nicholas Nickleby     1839--   967.0   
3      12                         The Old Curiosity Shop     1840--   700.0   
4      28  Barnaby Rudge: A Tale of the Riots of 'Eighty         --   917.0   
5      35                              Martin Chuzzlewit     1844--   968.0   
6      19                                 Dombey and Son     1848--   821.0   
7      14                              David Copperfield     1849--   766.0   
8      36                                    Bleak House     1853--  1023.0   
9      15                                     Hard Times     1854--   786.0   
10     33                                  Little Dorrit     1855--   963.0   
11      1                           A Tale of Two Ci

In [119]:
def writeFiles(df, author): 
    for i, row in df.iterrows():
        title = row.title
        if title.startswith('The'):
           title = title[4:]
        if title.startswith('A'):
           title = title[3:]
        if ':' in title:
           title = title.split(':')[0]
        title = row.title.lower().replace(' ','-')
        year = row.wp_pubDate
        filename = "%s/%s-%s-%s.txt" % (author, author, row.wp_pubDate, title)
        with open(filename, 'w') as f:
             f.write(row.text)

In [146]:
writeFiles(dNovelsMeta, 'dickens')

In [10]:
collinsMeta = getMeta(authors[1])

In [11]:
collinsMeta.title

0                                         The Moonstone
1         The Haunted Hotel: A Mystery of Modern Venice
2                                    The Woman in White
3                                               No Name
4                                          Man and Wife
5                                        The Black Robe
6                                        A Rogue's Life
7                                         Miss or Mrs.?
8                                  The Law and the Lady
9                                      The New Magdalen
10                                    The Two Destinies
11                                      The Frozen Deep
12                                           After Dark
13                    The Evil Genius: A Domestic Story
14                                      My Lady's Money
15                                           "I Say No"
16                                        Little Novels
17                                             A

In [13]:
collinsWP = pd.Series({'Basil': 1852, 'Hide and Seek': 1854, 'The Dead Secret': 1856, 'The Woman in White': 1860,
'Armadale': 1866, 'No Thoroughfare': 1867, 'The Moonstone': 1868, 'Man and Wife': 1870, 'Poor Miss Finch': 1872, 'The Dead Alive': 1874, 'The Law and the Lady': 1875,
'The Black Robe': 1881, 'Blind Love': 1889})

In [71]:
collinsWP

Armadale                1866
Basil                   1852
Blind Love              1889
Hide and Seek           1854
Man and Wife            1870
No Thoroughfare         1867
Poor Miss Finch         1872
The Black Robe          1881
The Dead Alive          1874
The Dead Secret         1856
The Law and the Lady    1875
The Moonstone           1868
The Woman in White      1860
dtype: int64

In [86]:
def lookup(title):
    return collinsMeta[collinsMeta.title.str.contains(title)]['id'].item()

In [87]:
lookup('Basil')

'4605.0'

In [74]:
collinsWP

Armadale                1866
Basil                   1852
Blind Love              1889
Hide and Seek           1854
Man and Wife            1870
No Thoroughfare         1867
Poor Miss Finch         1872
The Black Robe          1881
The Dead Alive          1874
The Dead Secret         1856
The Law and the Lady    1875
The Moonstone           1868
The Woman in White      1860
dtype: int64

In [100]:
for title, year in collinsWP.iteritems():
    print(title, lookup(title))

Armadale 1895.0
Basil 4605.0
Blind Love 7890.0
Hide and Seek 7893.0
Man and Wife 1586.0


ValueError: can only convert an array of size 1 to a Python scalar

In [76]:
collinsWPDF = pd.DataFrame(collinsWP)

In [92]:
collinsWPDF.drop(5)

,0,title,wp_pubYear
0,1866,Armadale,1866
1,1852,Basil,1852
2,1889,Blind Love,1889
3,1854,Hide and Seek,1854
4,1870,Man and Wife,1870
6,1872,Poor Miss Finch,1872
7,1881,The Black Robe,1881
8,1874,The Dead Alive,1874
9,1856,The Dead Secret,1856
10,1875,The Law and the Lady,1875


In [79]:
collinsWPDF['title'] = collinsWPDF.index

In [80]:
collinsWPDF

,0,title
Armadale,1866,Armadale
Basil,1852,Basil
Blind Love,1889,Blind Love
Hide and Seek,1854,Hide and Seek
Man and Wife,1870,Man and Wife
No Thoroughfare,1867,No Thoroughfare
Poor Miss Finch,1872,Poor Miss Finch
The Black Robe,1881,The Black Robe
The Dead Alive,1874,The Dead Alive
The Dead Secret,1856,The Dead Secret


In [93]:
collinsWPDF = collinsWPDF.reset_index()

In [82]:
collinsWPDF['wp_pubYear'] = collinsWPDF[0]

In [83]:
collinsWPDF = collinsWPDF.drop('index', axis=1)

In [97]:
collinsWPDF = collinsWPDF.drop(5)

In [98]:
collinsWPDF

,index,0,title,wp_pubYear
0,0,1866,Armadale,1866
1,1,1852,Basil,1852
2,2,1889,Blind Love,1889
3,3,1854,Hide and Seek,1854
4,4,1870,Man and Wife,1870
6,6,1872,Poor Miss Finch,1872
7,7,1881,The Black Robe,1881
8,8,1874,The Dead Alive,1874
9,9,1856,The Dead Secret,1856
10,10,1875,The Law and the Lady,1875


In [103]:
collinsWPDF['id'] = collinsWPDF.title.apply(lookup)

In [106]:
collinsWPDF['text'] = collinsWPDF['id'].apply(getText)

In [109]:
collinsWPDF['wp_pubDate'] = collinsWPDF['wp_pubYear']

In [113]:
collinsWPDF.wp_pubDate

0     1866
1     1852
2     1889
3     1854
4     1870
6     1872
7     1881
8     1874
9     1856
10    1875
11    1868
12    1860
Name: wp_pubDate, dtype: int64

In [121]:
writeFiles(collinsWPDF, 'collins')

In [122]:
authors[3]

'Meredith, George'

In [124]:
meredithMeta = getMeta(authors[3])

In [129]:
meredithCSV = pd.read_csv('meredith.csv')

In [130]:
meredithCSV

,title,year
0,The Shaving of Shagpat,1856
1,Farina,1857
2,The Ordeal of Richard Feverel,1859
3,Evan Harrington,1861
4,Emilia in England,1864
5,Rhoda Fleming,1865
6,Vittoria,1867
7,The Adventures of Harry Richmond,1871
8,Beauchamp's Career,1875
9,The House on the Beach,1877


In [157]:
def meredithLookup(title):
    meta = meredithMeta[(meredithMeta.title.str.contains(title))]
    if len(meta) > 1: 
        complete = meta[meredithMeta.title.str.contains('Complete')]
        if len(complete) > 0: 
            meta = complete
        else: 
            meta = meta.loc[0]
    if len(meta) == 0: 
        return None
    metaID = meta.id
    metaTitle = meta.title
    print(title, '---', metaID, metaTitle, '\n\n')
    return metaID.item()

In [158]:
meredithCSV['id'] = meredithCSV.title.apply(meredithLookup)

The Shaving of Shagpat --- 10    4405.0
Name: id, dtype: object 10    The Shaving of Shagpat; an Arabian entertainme...
Name: title, dtype: object 


Farina --- 97    4492.0
Name: id, dtype: object 97    Farina
Name: title, dtype: object 


The Ordeal of Richard Feverel --- 17    4412.0
Name: id, dtype: object 17    The Ordeal of Richard Feverel — Complete
Name: title, dtype: object 


Evan Harrington --- 39    4434.0
Name: id, dtype: object 39    Evan Harrington — Complete
Name: title, dtype: object 


Emilia in England --- 25    4420.0
Name: id, dtype: object 25    Sandra Belloni (originally Emilia in England) ...
Name: title, dtype: object 


Rhoda Fleming --- 31    4426.0
Name: id, dtype: object 31    Rhoda Fleming — Complete
Name: title, dtype: object 


Vittoria --- 48    4443.0
Name: id, dtype: object 48    Vittoria — Complete
Name: title, dtype: object 


The Adventures of Harry Richmond --- 57    4452.0
Name: id, dtype: object 57    The Adventures of Harry Richmond — Complete


/home/jon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [159]:
meredithCSV

,title,year,id
0,The Shaving of Shagpat,1856,4405.0
1,Farina,1857,4492.0
2,The Ordeal of Richard Feverel,1859,4412.0
3,Evan Harrington,1861,4434.0
4,Emilia in England,1864,4420.0
5,Rhoda Fleming,1865,4426.0
6,Vittoria,1867,4443.0
7,The Adventures of Harry Richmond,1871,4452.0
8,Beauchamp's Career,1875,4460.0
9,The House on the Beach,1877,4495.0


In [165]:
meredithCSV.set_value(15, 'id', meredithMeta.loc[81].id)

,title,year,id
0,The Shaving of Shagpat,1856,4405.0
1,Farina,1857,4492.0
2,The Ordeal of Richard Feverel,1859,4412.0
3,Evan Harrington,1861,4434.0
4,Emilia in England,1864,4420.0
5,Rhoda Fleming,1865,4426.0
6,Vittoria,1867,4443.0
7,The Adventures of Harry Richmond,1871,4452.0
8,Beauchamp's Career,1875,4460.0
9,The House on the Beach,1877,4495.0


In [169]:
meredithCSV['text'] = meredithCSV.id.apply(getText)

In [174]:
meredithCSV['wp_pubDate'] = meredithCSV['year']

In [175]:
writeFiles(meredithCSV, 'meredith')

In [177]:
authors[4]

'Conrad, Joseph'

In [178]:
conradMeta = getMeta(authors[4])

In [179]:
conradMeta

,,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,languages,lcsh,...,gr_image_url,gr_small_image_url,am_title,am_author,am_cat,am_cat_ancestors,am_genre,am_editorial_reviews,am_sales_rank,am_isbn
0,212,{'PR'},"Conrad, Joseph",1857,1924,5481,{'text/html; charset=utf-8': 'http://www.guten...,219.0,['en'],"{'Psychological fiction', 'Trading posts -- Fi...",...,https://images.gr-assets.com/books/1392799983m...,https://images.gr-assets.com/books/1392799983s...,Heart of Darkness (Dover Thrift Editions),Joseph Conrad,Classics,[<Element {http://webservices.amazon.com/AWSEC...,,"['<div> <div> <P>Although Polish by birth, Jos...",1416,0486264645
1,213,{'PR'},"Conrad, Joseph",1857,1924,339,{'text/plain; charset=utf-8': 'http://www.gute...,220.0,['en'],"{'Fugitives from justice -- Fiction', 'Psychol...",...,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,Heart of Darkness and The Secret Sharer (Banta...,Joseph Conrad,Classics,[<Element {http://webservices.amazon.com/AWSEC...,,['<i>Heart Of Darkness</i>. The story of the c...,180198,0553212141
2,442,{'PR'},"Conrad, Joseph",1857,1924,189,{'text/plain; charset=utf-8': 'http://www.gute...,451.0,['en'],"{'Sea stories', 'Ship captains -- Fiction'}",...,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,The Shadow-Line: A Confession (The Cambridge E...,Joseph Conrad,Classics,[<Element {http://webservices.amazon.com/AWSEC...,,"[""Joseph Conrad's short novel The Shadow-Line:...",1922208,1107024420
3,484,{'PR'},"Conrad, Joseph",1857,1924,27,{'text/html; charset=utf-8': 'http://www.guten...,493.0,['en'],"{'Man-woman relationships -- Fiction', 'Seafar...",...,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,Falk: A Reminiscence,Joseph Conrad,Literary,[<Element {http://webservices.amazon.com/AWSEC...,,['Falk: A Reminiscence is a humorous engaging ...,8541480,1542575060
4,485,{'PR'},"Conrad, Joseph",1857,1924,19,{'text/html; charset=utf-8': 'http://www.guten...,494.0,['en'],"{'Man-woman relationships -- Fiction', 'Ship c...",...,https://images.gr-assets.com/books/1423751323m...,https://images.gr-assets.com/books/1423751323s...,To-morrow (Penguin Little Black Classics) by J...,Joseph Conrad,Subjects,[<Element {http://webservices.amazon.com/AWSEC...,,[],None,
5,486,{'PR'},"Conrad, Joseph",1857,1924,101,{'text/plain; charset=utf-8': 'http://www.gute...,495.0,['en'],{'Man-woman relationships -- Fiction'},...,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,Amy Foster,Joseph Conrad,Antiques & Collectibles,[<Element {http://webservices.amazon.com/AWSEC...,,['Amy Foster'],2580252,1470157535
6,516,{'PR'},"Conrad, Joseph",1857,1924,127,{'text/plain; charset=utf-8': 'http://www.gute...,525.0,['en'],"{'Fiction', 'Sea stories', 'Adventure stories'}",...,https://images.gr-assets.com/books/1475304733m...,https://images.gr-assets.com/books/1475304733s...,Youth A Narrative,Joseph Conrad,Classics,[<Element {http://webservices.amazon.com/AWSEC...,,['This anthology is a thorough introduction to...,1360553,1466200448
7,517,{'PR'},"Conrad, Joseph",1857,1924,1962,{'text/html': 'http://www.gutenberg.org/ebooks...,526.0,['en'],"{'Psychological fiction', 'Trading posts -- Fi...",...,https://images.gr-assets.com/books/1392799983m...,https://images.gr-assets.com/books/1392799983s...,Heart of Darkness (Dover Thrift Editions),Joseph Conrad,Classics,[<Element {http://webservices.amazon.com/AWSEC...,,"['<div> <div> <P>Although Polish by birth, Jos...",1416,0486264645
8,518,{'PR'},"Conrad, Joseph",1857,1924,40,{'text/html; charset=utf-8': 'http://www.guten...,527.0,['en'],"{'Older men -- Fiction', 'Ship captains -- Fic...",...,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,Youth; Heart of Darkness; The End of the Tethe...,Joseph Conrad,Classics,[<Element {http://

In [181]:
conradCSV = pd.read_csv('conrad.csv')

In [182]:
conradCSV

,title,wp_pubDate
0,Almayer's Folly,1895
1,An Outcast of the Islands,1896
2,The Nigger of the Narcissus,1897
3,Heart of Darkness,1899
4,Lord Jim,1900
5,Typhoon,1902
6,The End of the Tether,1902
7,Nostromo,1904
8,The Secret Agent,1907
9,Under Western Eyes,1911


In [194]:
def conradLookup(title):
    meta = conradMeta[(conradMeta.title.str.contains(title))]
    if len(meta) > 1: 
        complete = meta[conradMeta.title.str.contains('Complete')]
        if len(complete) > 0: 
            meta = complete
        else: 
            try: 
                meta = meta.reset_index().loc[0]
            except: 
                pass
    if len(meta) == 0: 
        return None
    metaID = meta.id
    metaTitle = meta.title
    print(title, '---', metaID, metaTitle, '\n\n')
    try: 
        return metaID.item()
    except: 
        return metaID

In [196]:
conradCSV['id'] = conradCSV.title.apply(conradLookup)

Almayer's Folly --- 11    720.0
Name: id, dtype: object 11    Almayer's Folly: A Story of an Eastern River
Name: title, dtype: object 


An Outcast of the Islands --- 9    638.0
Name: id, dtype: object 9    An Outcast of the Islands
Name: title, dtype: object 


Heart of Darkness --- 219.0 Heart of Darkness 


Lord Jim --- 26    5658.0
Name: id, dtype: object 26    Lord Jim
Name: title, dtype: object 


Typhoon --- 17    1142.0
Name: id, dtype: object 17    Typhoon
Name: title, dtype: object 


The End of the Tether --- 8    527.0
Name: id, dtype: object 8    The End of the Tether
Name: title, dtype: object 


Nostromo --- 23    2021.0
Name: id, dtype: object 23    Nostromo: A Tale of the Seaboard
Name: title, dtype: object 


The Secret Agent --- 12    974.0
Name: id, dtype: object 12    The Secret Agent: A Simple Tale
Name: title, dtype: object 


Under Western Eyes --- 25    2480.0
Name: id, dtype: object 25    Under Western Eyes
Name: title, dtype: object 


Chance --- 1476.0 Chanc

/home/jon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [197]:
conradCSV

,title,wp_pubDate,id
0,Almayer's Folly,1895,720.0
1,An Outcast of the Islands,1896,638.0
2,The Nigger of the Narcissus,1897,None
3,Heart of Darkness,1899,219.0
4,Lord Jim,1900,5658.0
5,Typhoon,1902,1142.0
6,The End of the Tether,1902,527.0
7,Nostromo,1904,2021.0
8,The Secret Agent,1907,974.0
9,Under Western Eyes,1911,2480.0


In [201]:
conradMeta[conradMeta.title.str.contains('Suspense')].id

Series([], Name: id, dtype: object)

In [199]:
conradCSV.set_value(2, 'id', 17731.)

,title,wp_pubDate,id
0,Almayer's Folly,1895,720.0
1,An Outcast of the Islands,1896,638.0
2,The Nigger of the Narcissus,1897,17731
3,Heart of Darkness,1899,219.0
4,Lord Jim,1900,5658.0
5,Typhoon,1902,1142.0
6,The End of the Tether,1902,527.0
7,Nostromo,1904,2021.0
8,The Secret Agent,1907,974.0
9,Under Western Eyes,1911,2480.0


In [203]:
conradCSV['text'] = conradCSV.id.apply(getText)

In [204]:
conradCSV

,title,wp_pubDate,id,text
0,Almayer's Folly,1895,720.0,\n\n\n\n\nTranscribed from the 1915 T. Fisher ...
1,An Outcast of the Islands,1896,638.0,\n\n\n\n\nAN OUTCAST OF THE ISLANDS\n\nby Jose...
2,The Nigger of the Narcissus,1897,17731,\n\n\n\n\nTHE NIGGER of THE NARCISSUS\n\nA TAL...
3,Heart of Darkness,1899,219.0,\n\n\n\n\nHEART OF DARKNESS\n\nBy Joseph Conra...
4,Lord Jim,1900,5658.0,\n\n\n\n\nLORD JIM\n\nBY JOSEPH CONRAD\n\n\n\n...
5,Typhoon,1902,1142.0,\n\n\n\n\n[The other stories included in this ...
6,The End of the Tether,1902,527.0,\n\n\n\n\nTHE END OF THE TETHER\n\n\nBy Joseph...
7,Nostromo,1904,2021.0,\n\n\n\n\nNOSTROMO\n\nA TALE OF THE SEABOARD\n...
8,The Secret Agent,1907,974.0,\n\nTranscribed from the 1907 Methuen & Co edi...
9,Under Western Eyes,1911,2480.0,\n\n\n\n\nUNDER WESTERN EYES\n\nby JOSEPH CONR...


In [207]:
writeFiles(conradCSV.loc[:14], 'conrad')

In [208]:
authors[5]

'Cather, Willa'

In [228]:
catherMeta = getMeta(authors[5])

In [229]:
catherMeta.title

0                                    O Pioneers!
1                           The Song of the Lark
2                             Alexander's Bridge
3                                     My Antonia
4         The Troll Garden, and Selected Stories
5                                    One of Ours
6                    Youth and the Bright Medusa
7                                     My Ántonia
8    A Collection of Stories, Reviews and Essays
Name: title, dtype: object

In [214]:
def getAusMeta(author): 
    query = 'select * from pgaus where author="%s"' % author
    database = "sqlite:////home/jon/Code/gitenberg-scrape/pgaus.db"
    result = pd.read_sql(query, database)
    return result

In [227]:
catherAusMeta = getAusMeta('Willa Cather')

In [221]:
catherMeta

,index,DateAdded,Title and Author,IDa,IDb,URLOther,URLs,htmlURL,Title,txtURL,Author,Notes,Subtitle,ShortTitle,AuthorAlt,Text,HTML
0,904,Jul 2013,"The Enchanted Bluff, Willa Cather",[130437xx.xxx],2894A,http://gutenberg.net.au/ebooks13/1304371h.html,http://gutenberg.net.au/ebooks13/1304371h.html,http://gutenberg.net.au/ebooks13/1304371h.html,The Enchanted Bluff,None,Willa Cather,None,None,None,None,None,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1,2432,Nov 2006,"The Professor's House, Willa Cather",[060849xx.xxx],1367A,http://gutenberg.net.au/ebooks06/0608491.txt o...,http://gutenberg.net.au/ebooks06/0608491.txt o...,None,The Professor's House,http://gutenberg.net.au/ebooks06/0608491.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
2,3283,May 2005,"Not Under Forty, by Willa Cather",[050044xx.xxx],0439A,http://gutenberg.net.au/ebooks05/0500441.txt o...,http://gutenberg.net.au/ebooks05/0500441.txt o...,http://gutenberg.net.au/ebooks05/0500441h.html,Not Under Forty,http://gutenberg.net.au/ebooks05/0500441.txt,Willa Cather,None,None,None,None,"ï»¿<table width=""45%"" border =""0"">\r\n<tr>\r\n...",None
3,3295,Apr 2005,"My Mortal Enemy, by Willa Cather",[050032xx.xxx],0427A,http://gutenberg.net.au/ebooks05/0500321.txt o...,http://gutenberg.net.au/ebooks05/0500321.txt o...,http://gutenberg.net.au/ebooks05/0500321h.html,My Mortal Enemy,http://gutenberg.net.au/ebooks05/0500321.txt,Willa Cather,None,None,None,None,"ï»¿<table width=""45%"" border =""0"">\r\n<tr>\r\n...",None
4,3575,Dec 2002,"Obscure Destinies, by Willa Cather",[020113xx.xxx],0147A,http://gutenberg.net.au/ebooks02/0201131.txt o...,http://gutenberg.net.au/ebooks02/0201131.txt o...,None,Obscure Destinies,http://gutenberg.net.au/ebooks02/0201131.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
5,3605,Nov 2002,"The Professor's House, by Willa Cather",[020083xx.xxx],0117A,http://gutenberg.net.au/ebooks02/0200831.txt o...,http://gutenberg.net.au/ebooks02/0200831.txt o...,None,The Professor's House,http://gutenberg.net.au/ebooks02/0200831.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
6,3612,Oct 2002,"Shadows on the Rock, by Willa Cather",[020076xx.xxx],0110A,http://gutenberg.net.au/ebooks02/0200761.txt o...,http://gutenberg.net.au/ebooks02/0200761.txt o...,None,Shadows on the Rock,http://gutenberg.net.au/ebooks02/0200761.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
7,3639,Jul 2002,"Death Comes for the Archbishop, by Willa Cather",[020049xx.xxx],0083A,http://gutenberg.net.au/ebooks02/0200491.txt o...,http://gutenberg.net.au/ebooks02/0200491.txt o...,http://gutenberg.net.au/ebooks02/0200491h.html,Death Comes for the Archbishop,http://gutenberg.net.au/ebooks02/0200491.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
8,3640,Jul 2002,"Lucy Gayheart, by Willa Cather",[020048xx.xxx],0082A,http://gutenberg.net.au/ebooks02/0200481.txt o...,http://gutenberg.net.au/ebooks02/0200481.txt o...,http://gutenberg.net.au/ebooks02/0200481h.html,Lucy Gayheart,http://gutenberg.net.au/ebooks02/0200481.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None
9,3642,Jul 2002,"Sapphira and the Slave Girl, Willa Cather",[020046xx.xxx],0080A,http://gutenberg.net.au/ebooks02/0200461.txt o...,http://gutenberg.net.au/ebooks02/0200461.txt o...,http://gutenberg.net.au/ebooks02/0200461h.html,Sapphira and the Slave Girl,http://gutenberg.net.au/ebooks02/0200461.txt,Willa Cather,None,None,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n...",None


In [219]:
catherCSV = pd.read_csv('cather.csv')

In [220]:
catherCSV

,title,wp_pubDate
0,Alexander's Bridge,1912
1,O Pioneers!,1913
2,The Song of the Lark,1915
3,My Ántonia,1918
4,One of Ours,1922
5,A Lost Lady,1923
6,The Professor's House,1925
7,My Mortal Enemy,1926
8,Death Comes for the Archbishop,1927
9,Shadows on the Rock,1931


In [224]:
def catherLookup(title): 
    return catherMeta[catherMeta.Title == title]

In [225]:
catherLookup("Alexander's Bridge")

,index,DateAdded,Title and Author,IDa,IDb,URLOther,URLs,htmlURL,Title,txtURL,Author,Notes,Subtitle,ShortTitle,AuthorAlt,Text,HTML


In [231]:
def catherLookup(title):
    meta = catherMeta[(catherMeta.title.str.contains(title))]
    if len(meta) > 1: 
        complete = meta[catherMeta.title.str.contains('Complete')]
        if len(complete) > 0: 
            meta = complete
        else: 
            try: 
                meta = meta.reset_index().loc[0]
            except: 
                pass
    if len(meta) == 0: 
        return None
    metaID = meta.id
    metaTitle = meta.title
    print(title, '---', metaID, metaTitle, '\n\n')
    try: 
        return metaID.item()
    except: 
        return metaID

In [233]:
catherCSV['pgID'] = catherCSV.title.apply(catherLookup)

Alexander's Bridge --- 2    94.0
Name: id, dtype: object 2    Alexander's Bridge
Name: title, dtype: object 


O Pioneers! --- 0    24.0
Name: id, dtype: object 0    O Pioneers!
Name: title, dtype: object 


The Song of the Lark --- 1    44.0
Name: id, dtype: object 1    The Song of the Lark
Name: title, dtype: object 


My Ántonia --- 7    19810.0
Name: id, dtype: object 7    My Ántonia
Name: title, dtype: object 


One of Ours --- 5    2369.0
Name: id, dtype: object 5    One of Ours
Name: title, dtype: object 




In [234]:
catherCSV

,title,wp_pubDate,pgID
0,Alexander's Bridge,1912,94.0
1,O Pioneers!,1913,24.0
2,The Song of the Lark,1915,44.0
3,My Ántonia,1918,19810.0
4,One of Ours,1922,2369.0
5,A Lost Lady,1923,None
6,The Professor's House,1925,None
7,My Mortal Enemy,1926,None
8,Death Comes for the Archbishop,1927,None
9,Shadows on the Rock,1931,None


In [265]:
catherCSV['text'] = catherCSV.pgID.apply(getText)

In [258]:
def catherAusLookup(title): 
    results = catherAusMeta[catherAusMeta.Title == title]
    if len(results) == 1:
        return results.Text.item()
    else: 
        return None

In [267]:
catherCSV['ausText'] = catherCSV[5:].title.apply(catherAusLookup)

In [268]:
catherCSV

,title,wp_pubDate,pgID,text,ausText
0,Alexander's Bridge,1912,94.0,\n\n\n\n\nALEXANDER'S BRIDGE\n\nby Willa Cathe...,NaN
1,O Pioneers!,1913,24.0,\n\n\n\n\nO PIONEERS!\n\nby Willa Cather\n\n\n...,NaN
2,The Song of the Lark,1915,44.0,\n\n\n\n\nSONG OF THE LARK\n\nBy Willa Cather\...,NaN
3,My Ántonia,1918,19810.0,\n\n\n\n\nMy Antonia\n\nBy Willa Sibert Cather...,NaN
4,One of Ours,1922,2369.0,\n\n\n\n\nOne of Ours\n\nby Willa Cather\n\n\n...,NaN
5,A Lost Lady,1923,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."
6,The Professor's House,1925,None,None,None
7,My Mortal Enemy,1926,None,None,"ï»¿<table width=""45%"" border =""0"">\r\n<tr>\r\n..."
8,Death Comes for the Archbishop,1927,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."
9,Shadows on the Rock,1931,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."


In [270]:
catherCSV.set_value(6, 'ausText', catherAusMeta.loc[1]['Text'])

,title,wp_pubDate,pgID,text,ausText
0,Alexander's Bridge,1912,94.0,\n\n\n\n\nALEXANDER'S BRIDGE\n\nby Willa Cathe...,NaN
1,O Pioneers!,1913,24.0,\n\n\n\n\nO PIONEERS!\n\nby Willa Cather\n\n\n...,NaN
2,The Song of the Lark,1915,44.0,\n\n\n\n\nSONG OF THE LARK\n\nBy Willa Cather\...,NaN
3,My Ántonia,1918,19810.0,\n\n\n\n\nMy Antonia\n\nBy Willa Sibert Cather...,NaN
4,One of Ours,1922,2369.0,\n\n\n\n\nOne of Ours\n\nby Willa Cather\n\n\n...,NaN
5,A Lost Lady,1923,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."
6,The Professor's House,1925,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."
7,My Mortal Enemy,1926,None,None,"ï»¿<table width=""45%"" border =""0"">\r\n<tr>\r\n..."
8,Death Comes for the Archbishop,1927,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."
9,Shadows on the Rock,1931,None,None,"\r\n\r\nï»¿<table width=""45%"" border =""0"">\r\n..."


In [272]:
writeFiles(catherCSV[:5], 'cather')

In [275]:
def writeFilesAus(df, author): 
    for i, row in df.iterrows():
        title = row.title
        if title.startswith('The'):
            title = title[4:]
        if title.startswith('A'):
            title = title[3:]
        if ':' in title:
            title = title.split(':')[0]
        title = row.title.lower().replace(' ','-')
        year = row.wp_pubDate
        filename = "%s/%s-%s-%s.txt" % (author, author, row.wp_pubDate, title)
        with open(filename, 'w') as f:
             f.write(row.ausText)

In [276]:
writeFilesAus(catherCSV[5:], 'cather')